# Новые пары

Вас попросили написать маленькую, но важную функцию, которая дополняет матчи, найденные моделью.

Если товары 1 и 2 – матчи, а также товары 7 и 2 – матчи, то логично сделать вывод, что 1 и 7 тоже матч.
! В данном случае мы не разделяем товары на наши и и товары конкурентов!

Напишите функцию, которая выполняет описанный постпроцессинг предсказаний:

На входе и выходе ожидается список из tuple. Список на выходе должен быть отсортирован, не содержать дублей и в каждой паре на первом месте товар с меньшим индексом.

In [4]:
from typing import List
from typing import Tuple


def extend_matches(pairs: List[Tuple[int, int]]) -> List[Tuple[int, int]]:
    """Find matches"""
    pairs_t = pairs.copy()
    res_set = set()
    # result_list = []
    if pairs_t:  # Проверяем, что множество не пустое
        max_t = max(len(t) for t in pairs_t)
    else:
        return []

    while max_t > 0:
        for i in range(len(pairs_t)):
            for j in range(len(pairs_t)):
                if set(pairs_t[i]) & set(pairs_t[j]):
                    pairs_t.append(tuple(sorted(set(pairs_t[i]) | set(pairs_t[j]))))
        max_t -= 1
    sets = set(pairs_t)
    # print("s",sets)
    for tup in sets.copy():
        pairs = [(min(tup[i], tup[j]), max(tup[i], tup[j]))
                 for i in range(len(tup))
                 for j in range(i + 1, len(tup))]

        # Добавляем их во множество
        # sets.update(pairs)
        res_set.update(pairs)
    #
    # # Преобразуем множество в отсортированный список
    #     result_list = sorted(sets)
    # print("r",)

    return sorted(res_set)


In [5]:
extend_matches([(1, 2), (2, 3), (5, 3), (4, 6), (6, 7), (8, 9)])

KeyboardInterrupt: 

In [26]:
from typing import List
from typing import Tuple


def extend_matches(pairs: List[Tuple[int, int]]) -> List[Tuple[int, int]]:
    """Add new pairs based on existing ones"""
    # Collect all unique ids
    unique_ids = set()
    for x, y in pairs:
        unique_ids.add(x)
        unique_ids.add(y)

    # Create a dictionary of existing pairs
    pairs_dict = {x: {x} for x in unique_ids}
    for x, y in pairs:
        united_pairs = pairs_dict[x] | pairs_dict[y]
        for z in united_pairs:
            pairs_dict[z].update(united_pairs)

    # Create a list of new pairs
    new_pairs = []
    for x, x_pairs in pairs_dict.items():
        for y in x_pairs:
            new_pairs.append((x, y))


    # Drop self-pairs and duplicates
    new_pairs = [pair for pair in new_pairs if pair[0] < pair[1]]

    # Sort the list of new pairs
    new_pairs = sorted(new_pairs)

    return new_pairs


In [27]:
extend_matches([(1, 2), (2, 3), (5, 3), (4, 6), (6, 7), (8, 9)])

[(1, 2),
 (1, 3),
 (1, 5),
 (2, 3),
 (2, 5),
 (3, 5),
 (4, 6),
 (4, 7),
 (6, 7),
 (8, 9)]

# Match Groups
Отлично, работает как часы. Однако мы с командой посидели-подумали и пришли к выводу, что работать с парами не так удобно, как с группами матчей. Если товары (1, 2) и (7, 2) – матчи, почему бы не объединить их в одну группу (1, 2, 7)? Такой формат записи более удобен. Более того, из одних "источников правды" приходят данные в одном формате (группами), из других в формате пар (из модели). Формат match-групп кажется более универсальной записью.

Давайте обновим наш постпроцессинг, чтобы он работал как с парами, так и с тройками товаров, четвёрками и другими наборами товаров.

In [57]:
from typing import List


def extend_matches(groups: List[tuple]) -> List[tuple]:
    """Add new pairs based on existing ones"""
    # Collect all unique ids
    unique_ids = set()
    for x in groups:
        for y in x:
            unique_ids.add(y)

    # Create a dictionary of existing pairs
    pairs_dict = {x: {x} for x in unique_ids}
    for x in groups:
        for j in range(len(x)-1):
            united_pairs = pairs_dict[x[j]] | pairs_dict[x[j+1]]
            for z in united_pairs:
                pairs_dict[z].update(united_pairs)
    print(pairs_dict)
    # Create a list of new pairs

    new_pairs = [tuple(value) for value in pairs_dict.values()]


    # Drop self-pairs and duplicates
    new_pairs = {pair for pair in new_pairs if pair[0] < pair[1]}

    # Sort the list of new pairs
    new_pairs = sorted(new_pairs)

    return new_pairs

In [58]:
extend_matches([(5, 3, 4, 8), (1, 2), (7, 2)])

{1: {1, 2, 7}, 2: {1, 2, 7}, 3: {3, 4, 5, 8}, 4: {3, 4, 5, 8}, 5: {3, 4, 5, 8}, 7: {1, 2, 7}, 8: {3, 4, 5, 8}}


[(1, 2, 7), (3, 4, 5, 8)]

# Эталонное решение

In [60]:
from typing import List


def extend_matches(groups: List[tuple]) -> List[tuple]:
    """Extend and unite groups based on existing ones

    Example:
    input: [(1, 2), (2, 3), (5, 3), (4, 6), (6, 7), (8, 9)]
    output: [(1, 2, 3, 5), (4, 6, 7), (8, 9)]

    """
    groups = [set(group) for group in groups]
    new_groups = []

    for group in groups:
        for new_group in new_groups:
            if group & new_group:
                new_group |= group
                break
        else:
            new_groups.append(group)

    new_groups = [tuple(group) for group in new_groups]
    return new_groups


In [59]:
from typing import List
from typing import Tuple


def extend_matches(pairs: List[Tuple[int, int]]) -> List[Tuple[int, int]]:
    """Find matches"""
    pairs_t = pairs.copy()
    res_set = set()
    # result_list = []
    if pairs_t:  # Проверяем, что множество не пустое
        max_t = max(len(t) for t in pairs_t)
    else:
        return []

    while max_t > 0:
        for i in range(len(pairs_t)):
            for j in range(len(pairs_t)):
                if set(pairs_t[i]) & set(pairs_t[j]):
                    pairs_t.append(tuple(sorted(set(pairs_t[i]) | set(pairs_t[j]))))
        max_t -= 1
    sets = set(pairs_t)
    # print("s",sets)
    for tup in sets.copy():
        pairs = [(min(tup[i], tup[j]), max(tup[i], tup[j]))
                 for i in range(len(tup))
                 for j in range(i + 1, len(tup))]

        # Добавляем их во множество
        # sets.update(pairs)
        res_set.update(pairs)
    #
    # # Преобразуем множество в отсортированный список
    #     result_list = sorted(sets)
    # print("r",)

    return sorted(res_set)
